# Objective of this notebook

This jupyter notebook will help set up organisations and projects in Blue Brain Nexus.



## Step 1: Create and configure a Blue Brain Nexus client

The **nexus-python-sdk** is a Python wrapper for the Blue Brain Nexus REST API (https://github.com/BlueBrain/nexus-python-sdk/)

In [1]:
!pip install nexus-sdk

In [51]:
import nexussdk as nxs
import getpass
import json


Provide the URL of the **environment** as value for the deployment variable

In [2]:
DEPLOYMENT = "https://sandbox.bluebrainnexus.io/v1"

Provide your **token** as the value of the token variable below. Your token can be obtained (after log-in) by clicking on *Copy token* in the top left corner of the sandbox environment of Blue Brain Nexus: https://sandbox.bluebrainnexus.io/web/

In [ ]:
TOKEN = getpass.getpass()

Configure your environment and token by executing the code below

In [40]:
nxs.config.set_environment(DEPLOYMENT)

In [41]:
nxs.config.set_token(TOKEN)

To get more information about the github realm from within which you operate, you can fetch realm information as follows:

In [ ]:
nxs.realms.fetch("")

## Step 2: Create an organization


In [52]:
ORGANIZATION_LABEL = "myOrg" # Provide your organization label here

In [53]:
ORGANIZATION_DESCRIPTION = "The myOrg description" # Provide your organization description here

In [54]:
response = nxs.organizations.create(org_label=ORGANIZATION_LABEL, description=ORGANIZATION_DESCRIPTION)
utils.pretty_print(response)

{
    "@context": [
        "https://bluebrain.github.io/nexus/contexts/admin.json",
        "https://bluebrain.github.io/nexus/contexts/resource.json"
    ],
    "@id": "http://dev.nexus.ocp.bbp.epfl.ch/v1/orgs/myOrg",
    "@type": "Organization",
    "_uuid": "f9e9db8f-34e0-4d3c-b279-48e18d6edb77",
    "_rev": 1,
    "_deprecated": false,
    "_createdAt": "2019-07-22T11:40:56.869294Z",
    "_createdBy": "http://dev.nexus.ocp.bbp.epfl.ch/v1/realms/bbp/users/sy",
    "_updatedAt": "2019-07-22T11:40:56.869294Z",
    "_updatedBy": "http://dev.nexus.ocp.bbp.epfl.ch/v1/realms/bbp/users/sy"
}



## Step 3: Create and intialize a project

Projects belong to an organization and and they are used to :

* Group and categorize sub-resources.
* Define settings that apply for operations on all sub-resources.
* Provide isolation from resources inside other projects. This behavior can be changed by defining so-called resolvers.

Check out the documentation for more information: 

* https://bluebrainnexus.io/docs/api/index.html
* https://bluebrainnexus.io/docs/tutorial/getting-started/quick-start/index.html

To create a project, set a project label and description below. You may further define **base, vocab and api_mappings** (further information on those concepts can be found in the documentation; see links above).

In [55]:
import urllib.request, json
import utils

PROJECT_LABEL = "myProject" # Provide your project label here
PROJECT_DESCRIPTION = "myProject description" # Provide your project description here

# Vocab config
vocab = "%s/resources/%s/%s/_/"%(DEPLOYMENT, ORGANIZATION_LABEL, PROJECT_LABEL)
base = "https://bbp.epfl.ch/neurosciencegraph/data/"


#Nexus Api Mappings from https://raw.githubusercontent.com/BlueBrain/nexus-bbp-domains/master/mappings/nexus-api-mappings.json
nexus_api_mappings_file = "https://raw.githubusercontent.com/BlueBrain/nexus-bbp-domains/master/mappings/nexus-api-mappings.json"
 
with urllib.request.urlopen(nexus_api_mappings_file) as url:
    nexus_api_mappings = json.loads(url.read().decode())
    utils.pretty_print(nexus_api_mappings["apiMappings"])



[
    {
        "namespace": "https://neuroshapes.org/dash/ontology",
        "prefix": "ontologies"
    },
    {
        "namespace": "https://neuroshapes.org/dash/",
        "prefix": "datashapes"
    },
    {
        "namespace": "https://neuroshapes.org/dash/taxonomy",
        "prefix": "taxonomies"
    },
    {
        "namespace": "https://provshapes.org/commons/",
        "prefix": "provcommonshapes"
    },
    {
        "namespace": "http://schema.org/",
        "prefix": "schemaorg"
    },
    {
        "namespace": "https://incf.github.io/neuroshapes/contexts/",
        "prefix": "context"
    },
    {
        "namespace": "https://neuroshapes.org/commons/",
        "prefix": "commonshapes"
    },
    {
        "namespace": "https://provshapes.org/datashapes/",
        "prefix": "provdatashapes"
    },
    {
        "namespace": "http://www.w3.org/ns/prov#",
        "prefix": "prov"
    }
]



In [56]:
response = nxs.projects.create(org_label=ORGANIZATION_LABEL, project_label=PROJECT_LABEL, description=PROJECT_DESCRIPTION,
                                     api_mappings=nexus_api_mappings["apiMappings"], base=base, vocab=vocab)
utils.pretty_print(response)

{
    "@context": [
        "https://bluebrain.github.io/nexus/contexts/admin.json",
        "https://bluebrain.github.io/nexus/contexts/resource.json"
    ],
    "@id": "http://dev.nexus.ocp.bbp.epfl.ch/v1/projects/myOrg/myProject",
    "@type": "Project",
    "_uuid": "9b136b0a-5f1a-493b-93fd-f627e07f8efe",
    "_rev": 1,
    "_deprecated": false,
    "_createdAt": "2019-07-22T11:41:37.079329Z",
    "_createdBy": "http://dev.nexus.ocp.bbp.epfl.ch/v1/realms/bbp/users/sy",
    "_updatedAt": "2019-07-22T11:41:37.079329Z",
    "_updatedBy": "http://dev.nexus.ocp.bbp.epfl.ch/v1/realms/bbp/users/sy"
}



### Import the Blue Brain Knowledge Graph schema 

The following calls will create a jsonld context (identified by https://bbp.neuroshapes.org) resource importing the Blue Brain Knowledge Graph schema (identified by https://neuroshapes.org and available [here](https://sandbox.bluebrainnexus.io/web/neurosciencegraph/datamodels/resources/https%3A%2F%2Fneuroshapes.org)). The schema here is made of a vocabulary.

In [57]:
project_to_resolve_to = "/".join(["neurosciencegraph", "datamodels"])

# Resolver
cross_project_resolver = {
  "@id": base+"datamodelsresolver",
  "@type": [
    "CrossProject"
  ],
  "projects": [
    project_to_resolve_to
  ],
  "identities": [
    {
    "@type": "Authenticated",
    "realm": "bbp"
    }
  ],
  "priority": 50
}


response = utils.create_resolver(nxs,cross_project_resolver,ORGANIZATION_LABEL,PROJECT_LABEL)
utils.pretty_print(response)

{
    "@context": "https://bluebrain.github.io/nexus/contexts/resource.json",
    "@id": "https://bbp.epfl.ch/neurosciencegraph/data/datamodelsresolver",
    "@type": [
        "Resolver",
        "CrossProject"
    ],
    "_self": "http://dev.nexus.ocp.bbp.epfl.ch/v1/resolvers/myOrg/myProject/datamodelsresolver",
    "_constrainedBy": "https://bluebrain.github.io/nexus/schemas/resolver.json",
    "_project": "http://dev.nexus.ocp.bbp.epfl.ch/v1/projects/myOrg/myProject",
    "_rev": 1,
    "_deprecated": false,
    "_createdAt": "2019-07-22T11:41:45.731619Z",
    "_createdBy": "http://dev.nexus.ocp.bbp.epfl.ch/v1/realms/bbp/users/sy",
    "_updatedAt": "2019-07-22T11:41:45.731619Z",
    "_updatedBy": "http://dev.nexus.ocp.bbp.epfl.ch/v1/realms/bbp/users/sy",
    "_incoming": "http://dev.nexus.ocp.bbp.epfl.ch/v1/resolvers/myOrg/myProject/datamodelsresolver/incoming",
    "_outgoing": "http://dev.nexus.ocp.bbp.epfl.ch/v1/resolvers/myOrg/myProject/datamodelsresolver/outgoing"
}



In [58]:
# Context for data
bbp_context =  {
    "@context":[
        "https://neuroshapes.org",
        {
            "@vocab": vocab
        }
    ],
    "@id":"https://bbp.neuroshapes.org"
    
}

response = utils.create_resource(nxs,bbp_context,ORGANIZATION_LABEL,PROJECT_LABEL)
utils.pretty_print(response)

{
    "@context": "https://bluebrain.github.io/nexus/contexts/resource.json",
    "@id": "https://bbp.neuroshapes.org",
    "_self": "http://dev.nexus.ocp.bbp.epfl.ch/v1/resources/myOrg/myProject/_/https%3A%2F%2Fbbp.neuroshapes.org",
    "_constrainedBy": "https://bluebrain.github.io/nexus/schemas/unconstrained.json",
    "_project": "http://dev.nexus.ocp.bbp.epfl.ch/v1/projects/myOrg/myProject",
    "_rev": 1,
    "_deprecated": false,
    "_createdAt": "2019-07-22T11:41:49.714011Z",
    "_createdBy": "http://dev.nexus.ocp.bbp.epfl.ch/v1/realms/bbp/users/sy",
    "_updatedAt": "2019-07-22T11:41:49.714011Z",
    "_updatedBy": "http://dev.nexus.ocp.bbp.epfl.ch/v1/realms/bbp/users/sy",
    "_incoming": "http://dev.nexus.ocp.bbp.epfl.ch/v1/resources/myOrg/myProject/_/https%3A%2F%2Fbbp.neuroshapes.org/incoming",
    "_outgoing": "http://dev.nexus.ocp.bbp.epfl.ch/v1/resources/myOrg/myProject/_/https%3A%2F%2Fbbp.neuroshapes.org/outgoing"
}



## Step 4: Get ACLs of the project

The function below is a helper function to display the current ACL's

In [ ]:
def permissions_summary(subpath: str, filename: str = None, ancestors: bool = True):
    from collections import namedtuple
    import pandas as pd

    def clean(_id: str) -> str:
        base = "{}/".format(deployment)
        return _id.replace(base, "").replace("realms/", "")

    Row = namedtuple("Row", ["path", "identity", "permissions"])

    try:
        listing = nxs.acls.list(subpath, ancestors=ancestors, self=False)
    
    except nxs.HTTPError as e:
        print("<ERROR>")
        try:
            pp(e.response.json())
        except ValueError:
            print(e.response.text)
    
    else:
        if listing["_total"]:
            flattened = [Row(x["_path"], clean(y["identity"]["@id"]), y["permissions"])
                         for x in listing["_results"] for y in x["acl"]]

            tuples = [(x.path, x.identity) for x in flattened]
            data = [{y: "#" for y in x.permissions} for x in flattened]

            headers = pd.MultiIndex.from_tuples(tuples, names=["path", "identity"])

            df = pd.DataFrame(data, index=headers).transpose().sort_index(axis=0).fillna("")

            if filename:
                df.to_excel(filename)
                
            props = [("text-align", "center"), ("border", "1px  black solid !important")]
            
            return df.style.set_table_styles(
                [{"selector": "th", "props": props}, {"selector": "td", "props": props}])
        else:
            print("<NO PERMISSIONS TO DISPLAY>")
            if filename:
                print("<NO FILE WRITTEN>")

use this function to display the ACL's of your project:

In [ ]:
permissions_summary(f"{ORGANIZATION_LABEL}/{PROJECTLABEL}", ancestors=True)